In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

mengimport library yang dibutuhkan

In [ ]:
df = pd.read_excel('java_tourism.xlsx')
df.head()

,id,place_name,deskripsi,reviews_link,rating,total_reviews,address,city,province,gambar
0,1,4G Peak Gedangsari,4G Peak Gedangsari adalah puncak wisata alam y...,https://serpapi.com/search.json?data_id=0x2e7a...,4.4,353,"4HRC+3CV, Manggung, Ngalang, Gedang Sari, Gunu...",gunung kidul,yogyakarta,4G Peak Gedangsari.png
1,2,Ade Irma Suryani Nasution Traffic Garden,Ade Irma Suryani Nasution Traffic Garden adala...,https://serpapi.com/search.json?data_id=0x2e68...,4.4,13178,"Jl. Belitung No.1, Merdeka, Sumurbandung, Band...",bandung,jawa barat,Ade Irma Suryani Nasution Traffic Garden.jpg
2,3,Agro Tourism Bhumi Merapi,Agro Tourism Bhumi Merapi adalah destinasi wis...,https://serpapi.com/search.json?data_id=0x2e7a...,4.5,13960,"Jl. Kaliurang No.Km.20, Sawungan, Hargobinangu...",sleman,yogyakarta,Agro Tourism Bhumi Merapi.jpg
3,4,Air Mancur Menari Manahan,Air Mancur Menari Manahan adalah atraksi wisat...,https://serpapi.com/search.json?data_id=0x2e7a...,4.5,80,"Manahan, Kec. Banjarsari, Kota Surakarta, Jawa...",surakarta,jawa tengah,Air Mancur Menari Manahan.jpeg
4,5,Air Terjun Banyu Anjlok,Air Terjun Banyu Anjlok adalah air terjun yang...,https://serpapi.com/search.json?data_id=0x2e70...,4.5,240,"Somosari VI, Somosari, Kec. Batealit, Kabupate...",jepara,jawa tengah,Air Terjun Banyu Anjlok.png


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 868 entries, 0 to 867
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             868 non-null    int64  
 1   place_name     868 non-null    object 
 2   deskripsi      868 non-null    object 
 3   reviews_link   868 non-null    object 
 4   rating         868 non-null    float64
 5   total_reviews  868 non-null    int64  
 6   address        868 non-null    object 
 7   city           868 non-null    object 
 8   province       868 non-null    object 
 9   gambar         868 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 67.9+ KB


Dataset ini terdiri dari 868 entri yang mencakup informasi mengenai tempat wisata di Pulau Jawa, dengan 10 kolom yang meliputi id,place_name, deskripsi, reviews_link, rating, total_reviews, address, city, province, dan gambar

In [ ]:
keywords = {
    'alam': ['gunung', 'pantai', 'danau', 'air terjun', 'hutan', 'taman nasional', 'cagar alam', 'gua', 'pemandangan', 'alam', 'sungai', 'laut'],
    'buatan': ['taman hiburan', 'museum', 'monumen', 'tempat wisata', 'kebun binatang', 'waterpark', 'kolam renang', 'gedung', 'bangunan', 'buatan', 'buatan manusia', 'theme park'],
    'budaya': ['budaya', 'tradisional', 'adat', 'seni', 'pertunjukan', 'sejarah', 'kerajaan', 'festival', 'tari', 'musik', 'candi', 'batik', 'keraton'],
    'religi': ['masjid', 'gereja', 'pura', 'vihara', 'kuil', 'makam', 'ziarah', 'religi', 'agama', 'keramat', 'rohani'],
    'edukasi': ['edukasi', 'pendidikan', 'belajar', 'ilmu pengetahuan', 'sekolah', 'universitas', 'planetarium', 'observatorium', 'arkeologi', 'penelitian', 'sains', 'iptek']
}

membuat kata kunci untuk memberikan lebel tipe wisiata

In [ ]:
def tag_labels(description):
    description = description.lower()
    labels = {key: 0 for key in keywords.keys()}
    for label, kws in keywords.items():
        for kw in kws:
            if kw in description:
                labels[label] = 1
                break
    return labels

label_df = df['deskripsi'].apply(tag_labels).apply(pd.Series)
df = pd.concat([df, label_df], axis=1)

print("Data dengan label tipe wisata:")

Kode di atas berfungsi untuk mengklasifikasikan deskripsi tempat wisata ke dalam berbagai kategori seperti alam, buatan, budaya, religi, dan edukasi berdasarkan kata kunci yang ada dalam deskripsi. Setiap kali sebuah kata kunci ditemukan dalam deskripsi, label untuk kategori tersebut diberi nilai 1. Setelah itu, label-label ini ditambahkan ke dalam DataFrame, sehingga setiap tempat wisata memiliki kolom yang menunjukkan tipe wisata yang relevan dengan deskripsinya. Ini memudahkan sistem untuk mengidentifikasi dan mengategorikan tempat wisata secara otomatis.

In [ ]:
df.head()

,id,place_name,deskripsi,reviews_link,rating,total_reviews,address,city,province,gambar,alam,buatan,budaya,religi,edukasi
0,1,4G Peak Gedangsari,4G Peak Gedangsari adalah puncak wisata alam y...,https://serpapi.com/search.json?data_id=0x2e7a...,4.4,353,"4HRC+3CV, Manggung, Ngalang, Gedang Sari, Gunu...",gunung kidul,yogyakarta,4G Peak Gedangsari.png,1,0,0,0,0
1,2,Ade Irma Suryani Nasution Traffic Garden,Ade Irma Suryani Nasution Traffic Garden adala...,https://serpapi.com/search.json?data_id=0x2e68...,4.4,13178,"Jl. Belitung No.1, Merdeka, Sumurbandung, Band...",bandung,jawa barat,Ade Irma Suryani Nasution Traffic Garden.jpg,0,0,0,0,1
2,3,Agro Tourism Bhumi Merapi,Agro Tourism Bhumi Merapi adalah destinasi wis...,https://serpapi.com/search.json?data_id=0x2e7a...,4.5,13960,"Jl. Kaliurang No.Km.20, Sawungan, Hargobinangu...",sleman,yogyakarta,Agro Tourism Bhumi Merapi.jpg,0,0,0,0,1
3,4,Air Mancur Menari Manahan,Air Mancur Menari Manahan adalah atraksi wisat...,https://serpapi.com/search.json?data_id=0x2e7a...,4.5,80,"Manahan, Kec. Banjarsari, Kota Surakarta, Jawa...",surakarta,jawa tengah,Air Mancur Menari Manahan.jpeg,1,1,1,0,0
4,5,Air Terjun Banyu Anjlok,Air Terjun Banyu Anjlok adalah air terjun yang...,https://serpapi.com/search.json?data_id=0x2e70...,4.5,240,"Somosari VI, Somosari, Kec. Batealit, Kabupate...",jepara,jawa tengah,Air Terjun Banyu Anjlok.png,1,0,0,0,0


In [ ]:
# ----------------------------
# TOKENIZER & SEQUENCE PREPARATION
# ----------------------------

texts = df['deskripsi'].tolist()
labels = df[['alam', 'buatan', 'budaya', 'religi', 'edukasi']].values

max_words = 1000
max_len = 50

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
x_data = pad_sequences(sequences, maxlen=max_len)

Kode di atas mempersiapkan data teks dengan mengubah deskripsi tempat wisata menjadi urutan angka menggunakan Tokenizer dan membatasi jumlah kata hingga 1000 kata yang paling sering muncul. Kemudian, urutan angka tersebut dipadatkan agar memiliki panjang yang konsisten, yaitu 50 kata, sehingga siap digunakan dalam pelatihan model machine learning.

In [ ]:
# Split train-val (simple split)
train_size = int(len(x_data) * 0.8)
x_train, x_val = x_data[:train_size], x_data[train_size:]
y_train, y_val = labels[:train_size], labels[train_size:]

Kode di atas membagi data menjadi dua bagian: data pelatihan (train) dan data validasi (val). 80% dari data digunakan untuk pelatihan (x_train, y_train), sementara 20% sisanya digunakan untuk validasi (x_val, y_val)

In [ ]:
# ----------------------------
# MODEL BUILDING (LSTM)
# ----------------------------

input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=max_words, output_dim=64, input_length=max_len)(input_layer)

# LSTM layer
lstm_layer = LSTM(128, return_sequences=False)(embedding_layer)

# Optional Dropout layer to avoid overfitting
dropout_layer = Dropout(0.5)(lstm_layer)

# Dense layer for further representation
dense_layer = Dense(64, activation='relu')(dropout_layer)

# Output layer
output_layer = Dense(5, activation='sigmoid')(dense_layer)

# Create model
model_lstm = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the LSTM model
print(model_lstm.summary())


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 50, 64)         │        64,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 171,397 (669.52 KB)

 Trainable params: 171,397 (669.52 KB)

 Non-trainable params: 0 (0.00 B)

None


Kode di atas membangun model LSTM untuk klasifikasi teks dengan input berupa deskripsi tempat wisata. Model dimulai dengan embedding layer untuk representasi kata, dilanjutkan dengan LSTM layer untuk memproses urutan teks, dan dropout layer untuk mencegah overfitting. Kemudian, dense layer diikuti dengan output layer yang menggunakan fungsi aktivasi sigmoid untuk memprediksi 5 kategori tipe wisata. Model dikompilasi dengan binary crossentropy dan adam optimizer, serta menggunakan accuracy sebagai metrik evaluasi.

In [ ]:
# ----------------------------
# TRAINING (MODIFIKASI)
# ----------------------------

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# Training model with EarlyStopping
history_lstm = model_lstm.fit(
    x_train, y_train,
    epochs=10,  # Bisa ditingkatkan lebih tinggi
    batch_size=16,
    validation_data=(x_val, y_val),
    callbacks=[early_stop]
)


Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.7953 - loss: 0.1460 - val_accuracy: 0.8621 - val_loss: 0.2040
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.8088 - loss: 0.1383 - val_accuracy: 0.8103 - val_loss: 0.1903
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.7684 - loss: 0.1177 - val_accuracy: 0.8448 - val_loss: 0.1936
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8204 - loss: 0.1026 - val_accuracy: 0.8046 - val_loss: 0.1740
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.8164 - loss: 0.0995 - val_accuracy: 0.8333 - val_loss: 0.1980
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8182 - loss: 0.0887 - val_accuracy: 0.8103 - val_loss: 0.1841
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.7693 - loss: 0.0888 - val_accuracy: 0.8563 - val_loss: 0.1830
Epoch 7: early stopping
Restoring model weights from the end of the best epoch: 4.


Kode di atas melatih model LSTM dengan early stopping untuk menghentikan pelatihan jika val_loss tidak membaik selama 3 epoch. Pelatihan dilakukan selama 10 epoch dengan batch size 16, menggunakan data validasi untuk evaluasi.

In [ ]:
# Definisikan label_names
label_names = ['alam', 'buatan', 'budaya', 'religi', 'edukasi']

# ----------------------------
# EVALUASI MODEL
# ----------------------------

# Evaluasi model pada set validasi
loss, accuracy = model_lstm.evaluate(x_val, y_val, verbose=0)
print(f"\nEvaluasi Model LSTM pada Set Validasi:")
print(f"Loss: {loss:.4f}")
print(f"Akurasi: {accuracy:.4f}")

# Prediksi probabilitas pada set validasi
y_pred_probs_lstm = model_lstm.predict(x_val)

# Mengonversi probabilitas menjadi prediksi biner menggunakan ambang batas (misalnya, 0.5)
y_pred_bin_lstm = (y_pred_probs_lstm >= 0.5).astype(int)

# Laporan klasifikasi untuk model LSTM
print("\nLaporan Klasifikasi pada Set Validasi untuk Model LSTM:")
print(classification_report(y_val, y_pred_bin_lstm, target_names=label_names, zero_division=0))



Evaluasi Model LSTM pada Set Validasi:
Loss: 0.1740
Akurasi: 0.8046
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step

Laporan Klasifikasi pada Set Validasi untuk Model LSTM:
              precision    recall  f1-score   support

        alam       0.96      0.97      0.97       145
      buatan       0.98      0.84      0.90        55
      budaya       0.86      0.67      0.75        36
      religi       0.89      0.73      0.80        11
     edukasi       0.74      0.87      0.80        23

   micro avg       0.93      0.89      0.91       270
   macro avg       0.88      0.81      0.84       270
weighted avg       0.93      0.89      0.90       270
 samples avg       0.96      0.93      0.93       270



Kode di atas mengevaluasi model LSTM pada data validasi dengan menghitung loss dan akurasi, lalu mengonversi prediksi probabilitas menjadi prediksi biner. Laporan klasifikasi kemudian ditampilkan untuk masing-masing kategori tipe wisata, memberikan metrik seperti precision, recall, dan f1-score.








In [ ]:
# ----------------------------
# PREDIKSI LABEL DARI INPUT USER
# ----------------------------

def predict_labels(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len)
    preds = model_lstm.predict(padded)[0]
    labels_pred = {label: float(preds[i]) for i, label in enumerate(['alam', 'buatan', 'budaya', 'religi', 'edukasi'])}
    # Threshold 0.5 untuk keputusan 0/1
    labels_bin = {k: int(v >= 0.5) for k, v in labels_pred.items()}
    return labels_bin, labels_pred


Kode di atas mendefinisikan fungsi predict_labels yang mengubah input teks menjadi urutan angka, memprediksi probabilitas dengan model LSTM, dan mengonversinya menjadi prediksi biner menggunakan ambang batas 0.5. Fungsi ini mengembalikan hasil prediksi dalam bentuk biner dan probabilitas untuk setiap tipe wisata.

In [ ]:
# ----------------------------
# REKOMENDASI BERBASIS SIMILARITY DENGAN THRESHOLD
# ----------------------------

def recommend_places(user_province, user_labels, user_description, df, top_n=10, threshold=0.10):
    # Filter provinsi
    filtered_df = df[df['province'].str.lower() == user_province.lower()]
    if filtered_df.empty:
        return "Maaf, tidak ada tempat di provinsi tersebut."

    # Filter tipe wisata sesuai input user (minimal satu tipe match)
    selected_labels = [label for label, val in user_labels.items() if val == 1]
    if not selected_labels:
        return "Mohon pilih minimal satu tipe wisata."

    mask = filtered_df[selected_labels].sum(axis=1) > 0
    filtered_df = filtered_df[mask]
    if filtered_df.empty:
        return "Maaf, tidak ada tempat dengan tipe wisata tersebut di provinsi itu."

    # Vektorisasi dengan TF-IDF
    corpus = filtered_df['deskripsi'].tolist()
    corpus_with_query = corpus + [user_description]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus_with_query)

    cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()

    # Filter berdasarkan threshold
    top_indices = cosine_sim.argsort()[::-1]
    top_indices = [idx for idx in top_indices if cosine_sim[idx] >= threshold][:top_n]  # Terapkan threshold

    # Jika tidak ada tempat yang memenuhi threshold
    if len(top_indices) == 0:
        return "Maaf, tidak ada tempat dengan kemiripan yang cukup di provinsi tersebut."

    recommendations = filtered_df.iloc[top_indices]

    result = []
    for idx, score in zip(top_indices, cosine_sim[top_indices]):
        place = filtered_df.iloc[idx]
        result.append({
            'place_name': place['place_name'],
            'province': place['province'],
            'deskripsi': place['deskripsi'],
            'gambar': place['gambar'],
            'similarity_score': float(score)
        })

    return result

Kode di atas mendefinisikan fungsi recommend_places yang memberikan rekomendasi tempat wisata berdasarkan kemiripan deskripsi dengan input pengguna. Fungsi ini memfilter data berdasarkan provinsi dan tipe wisata, lalu menghitung cosine similarity antara deskripsi input dan dataset. Tempat dengan skor kemiripan di atas threshold akan dikembalikan sebagai rekomendasi. Jika tidak ada tempat yang cocok, pesan yang sesuai akan ditampilkan.

In [ ]:
# ----------------------------
# INTERAKSI USER (INPUT LANGSUNG)
# ----------------------------

print("\n==== SISTEM REKOMENDASI WISATA ====\n")

# Input langsung dari user
user_province = input("Masukkan provinsi tujuan wisata: ").strip()

tipe_input = input("Masukkan tipe wisata yang diinginkan (pisahkan dengan koma, contoh: alam, budaya, religi): ").strip().lower()
selected_labels_input = [tipe.strip() for tipe in tipe_input.split(",")]

user_input_desc = input("Masukkan ciri-ciri tempat wisata (kalimat bebas): ").strip()

# Validasi dan buat dictionary tipe wisata
all_labels = ['alam', 'buatan', 'budaya', 'religi', 'edukasi']
selected_labels = {label: 1 if label in selected_labels_input else 0 for label in all_labels}

# Cetak ringkasan input user
print("\n=== RINGKASAN INPUT USER ===")
print("Provinsi:", user_province)
print("Tipe wisata dipilih:", [k for k, v in selected_labels.items() if v == 1])
print("Ciri-ciri input:", user_input_desc)

# Prediksi tipe wisata dari deskripsi
user_labels_bin, user_labels_prob = predict_labels(user_input_desc)
print("\n=== Prediksi Sistem Berdasarkan Ciri-ciri ===")
print("Probabilitas Label:", {k: f"{v:.2f}" for k,v in user_labels_prob.items()})

# Jalankan rekomendasi
recommendations = recommend_places(user_province, selected_labels, user_input_desc, df, top_n=10)

# Tampilkan hasil
if isinstance(recommendations, str):
    print("\n" + recommendations)
else:
    print("\n=== Rekomendasi Tempat Wisata ===\n")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. Nama Tempat : {rec['place_name']}")
        print(f"   Provinsi    : {rec['province']}")
        print(f"   Deskripsi   : {rec['deskripsi']}")
        print(f"   Gambar      : {rec['gambar']}")
        print(f"   Skor Kemiripan: {rec['similarity_score']:.4f}")
        print("-" * 50)



==== SISTEM REKOMENDASI WISATA ====

Masukkan provinsi tujuan wisata: jawa barat
Masukkan tipe wisata yang diinginkan (pisahkan dengan koma, contoh: alam, budaya, religi): alam
Masukkan ciri-ciri tempat wisata (kalimat bebas): gunung yang indah

=== RINGKASAN INPUT USER ===
Provinsi: jawa barat
Tipe wisata dipilih: ['alam']
Ciri-ciri input: gunung yang indah
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step

=== Prediksi Sistem Berdasarkan Ciri-ciri ===
Probabilitas Label: {'alam': '0.23', 'buatan': '0.19', 'budaya': '0.92', 'religi': '0.70', 'edukasi': '0.11'}

=== Rekomendasi Tempat Wisata ===

1. Nama Tempat : Gunung Padang
   Provinsi    : jawa barat
   Deskripsi   : Gunung Padang adalah situs megalitikum prasejarah yang menawarkan wisata budaya dan sejarah yang mendalam. Pengunjung dapat menjelajahi peninggalan purbakala dan belajar tentang peradaban kuno yang pernah mendiami area ini.
   Gambar      : Gunung Padang.jpg
   Skor Kemiripan: 0.2022
----------------------------------------------

Kode di atas memungkinkan pengguna untuk memasukkan provinsi, tipe wisata, dan deskripsi tempat wisata. Sistem kemudian memvalidasi input, memprediksi tipe wisata berdasarkan deskripsi, dan memberikan rekomendasi tempat wisata yang sesuai dengan provinsi dan tipe yang dipilih. Hasil rekomendasi mencakup nama tempat, provinsi, deskripsi, gambar, dan skor kemiripan.

In [ ]:
model_lstm.save('multilabel_model.h5')

import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


Menyimpan model dan tokenizer

In [ ]:
df.to_excel('java.xlsx', index=False)


menyimpan dataframe ke excel

In [ ]:
from google.colab import files
files.download('java.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>